## Code to bring in and downsample the forcings

This code works off the xarray builder code first

In [ ]:
%matplotlib inline 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import time
import xarray as xr

#### Try getting one of the files as a test before trying to pull them all in

In [ ]:
# Define the path to all of the data you want to load
path = '/srv/shared/deep_stac/data/forcings/2017/*2017.nc'
# Make the path a global varaible and make a list of the files 
flist=glob.glob(path)
print(len(flist))

#### All of the forcings data is contained within a text file called "forcings_key.txt"

This file can be used to cycle through each netCDF input and stitch it together in the xarray

In [ ]:
!ls /home/jovyan/ghw2019_deepstac/notebooks/vermillion

In [ ]:
# Open and filter to the 'primary' forcings 
pth = '/home/jovyan/ghw2019_deepstac/notebooks/vermillion'
fname = 'forcings_key.txt'

i = 1
key = np.genfromtxt(os.path.join(pth,fname), dtype='str', delimiter=',')

len(key)

pared = key[key[:,1]=='primary']

# And the names of the files are in:
pared[:,0]

#### Define a function for renaming (on the fly) input datasets with duplicate names
(for example, inside the netCDF objects, both rmax and rmin were assigned a name of "relative_humidity")

In [ ]:
def rnm(ds):
    if ds.encoding['source'].startswith('tmmx'):
        ds.rename(dict(air_temperature='tmax'), inplace=True)
    elif ds.encoding['source'].startswith('tmmn'):
        ds.rename(dict(air_temperature='tmin'), inplace=True)
    elif ds.encoding['source'].startswith('rmax'):
        ds.rename(dict(relative_humidity='rmax'), inplace=True)        
    elif ds.encoding['source'].startswith('rmin'):
        ds.rename(dict(relative_humidity='rmin'), inplace=True)        


#### Load up the data into xarray

In [ ]:
pared[:,0]
for itm in pared[:,0]:
    print(itm)

In [ ]:
# This is the loation of the forcings data
pth = '/srv/shared/deep_stac/data/forcings'
yrs = ['2014', '2015', '2016', '2017']

all_files = []
for yr in yrs:
    for itm in pared[:,0]:
        # For reasons unbeknownst to the user, these two datasets aren't coming in.
        #if any(x in itm for x in ['rmin']):   #,'tmmx'
        #    continue
        all_files.append(os.path.join(pth,yr,itm.split('_')[0] + '_' + yr + '.nc'))

# merge all forcings data into a single xarray in one go.
ds = xr.open_mfdataset(all_files, preprocess=rnm, combine='by_coords')

#### Just for grins, here is how to visualize one of the datasets:

In [ ]:
# Working with one dataset is better than many for learning

path1 = '/srv/shared/deep_stac/data/forcings/2017/tmmx_2017.nc'
d1 = xr.open_dataset(path1)
# d1['day'][0]['air_temperature'].plot
d1
temp=d1.air_temperature
temp.isel(day=0).plot()
#day["air_temperature"][0].plot

#### And here is the guts of the xarray (should one be interested):

In [ ]:
!ls /srv/shared/deep_stac/data/forcings/2017/

In [ ]:
all_files